# Image Quality assessment using Deep Learning techniques

## Importing libraries

In [1]:
import numpy as np
import cv2

In [ ]:
import os

In [ ]:
# connecting google drive and colab
from google.colab import drive
drive.mount('/content/drive')

## SSIM code

In [ ]:
# Function to calculate SSIM and save SSIM map
def calculate_and_save_ssim_map(reference_img_path, distorted_img_path, output_path):
    # Load reference and distorted images
    img1 = cv2.imread(reference_img_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(distorted_img_path, cv2.IMREAD_GRAYSCALE)

    # Calculate SSIM and SSIM map
    mssim, ssim_map = cal_ssim(img1, img2)

    # Save the SSIM map
    cv2.imwrite(output_path, (ssim_map * 255).astype(np.uint8))

In [ ]:
import cv2
import numpy as np
from scipy import signal

def cal_ssim(img1, img2):

    K = [0.01, 0.03]
    L = 255
    kernelX = cv2.getGaussianKernel(11, 1.5)
    window = kernelX * kernelX.T

    M,N = np.shape(img1)

    C1 = (K[0]*L)**2
    C2 = (K[1]*L)**2
    img1 = np.float64(img1)
    img2 = np.float64(img2)

    mu1 = signal.convolve2d(img1, window, 'valid')
    mu2 = signal.convolve2d(img2, window, 'valid')

    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2


    sigma1_sq = signal.convolve2d(img1*img1, window, 'valid') - mu1_sq
    sigma2_sq = signal.convolve2d(img2*img2, window, 'valid') - mu2_sq
    sigma12 = signal.convolve2d(img1*img2, window, 'valid') - mu1_mu2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
    mssim = np.mean(ssim_map)
    return mssim,ssim_map

In [ ]:
dist_fold = '/content/drive/MyDrive/NRIQA/distorted_images'
ref_fold = '/content/drive/MyDrive/NRIQA/reference_images'

out_fold = '/content/drive/MyDrive/NRIQA/output'
os.makedirs(out_fold, exist_ok = True)

In [ ]:
distorted_files = os.listdir(dist_fold)
reference_files = os.listdir(ref_fold)

In [ ]:
for reference_file in reference_files:
    # Iterate through each distorted image
    for distorted_file in distorted_files:
        # Check if the first 3 letters of the distorted image match the reference image
        if reference_file[:3] == distorted_file[:3]:
            distorted_img_path = os.path.join(dist_fold, distorted_file)
            reference_img_path = os.path.join(ref_fold, reference_file)
            output_path = os.path.join(out_fold, f'{os.path.splitext(distorted_file)[0]}_ssim_map.bmp')

            # Calculate and save SSIM map for each pair of images
            calculate_and_save_ssim_map(reference_img_path, distorted_img_path, output_path)

## CAE

### pre-processing

In [ ]:
# checking whether the number of distortion images and distortion maps are equal or not
dist_img = '/content/drive/MyDrive/NRIQA/distorted_images'
dist_map = '/content/drive/MyDrive/NRIQA/output'

file1 = os.listdir(dist_img)
file2 = os.listdir(dist_map)
if len(file1) == len(file2):
  print('Both have same number of images')
else:
  print('Both doesnt have same number of images')

In [ ]:
print(file1)

In [ ]:
print(file2)

In [ ]:
# fixing image size
height = 200
width =  150
x = []
y = []

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
for i in file1:
  img_fold = os.path.join(dist_img, i)
  image = cv2.imread(img_fold)
  resized_image = cv2.resize(image, (height, width))
  im_arr = resized_image/255.0
  x.append(im_arr)

In [ ]:
x_array = np.array(x)

In [ ]:
x_array.shape

In [ ]:
for j in file2:
  map_fold = os.path.join(dist_map, j)
  map = cv2.imread(map_fold)
  resized_map = cv2.resize(map, (height, width))
  map_arr = resized_map/255.0
  y.append(map_arr)

In [ ]:
y_array = np.array(y)

In [ ]:
x_array.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
from keras.layers import Conv2D, UpSampling2D

In [ ]:
input_shape = (150, 200, 3)

In [ ]:
model = Sequential([

    # Encoder
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape = (input_shape)),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),

    # Decoder
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same'),
])

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])

In [ ]:
h = model.fit(x_array, y_array, epochs=10, batch_size=10, validation_split=0.2)

In [ ]:
h.save('/content/drive/MyDrive/NRIQA/model/cae_model.h5')

In [ ]:
from keras.models import load_model
loaded_model = load_model('/content/drive/MyDrive/NRIQA/model/cae_model.h5')

In [ ]:
import cv2
import numpy as np

new_image = cv2.imread('/content/drive/MyDrive/NRIQA/test/images.jpeg')
new_image = cv2.resize(new_image, (200, 200))
new_image = new_image / 255.0
new_image = np.expand_dims(new_image, axis=0)


In [ ]:
predicted_distortion_map = loaded_model.predict(new_image)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

new_image = cv2.imread('/content/drive/MyDrive/NRIQA/test/1.jpg')
height, width = 200, 200
new_image = cv2.resize(new_image, (width, height))
new_image = new_image / 255.0
new_image = np.expand_dims(new_image, axis=0)
predicted_distortion_map = loaded_model.predict(new_image)
predicted_distortion_map = (predicted_distortion_map * 255).astype(np.uint8)


plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(new_image[0])
plt.title('Tested Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(predicted_distortion_map[0, :, :, 0], cmap='gray')
plt.title('Distortion Map')
plt.axis('off')
plt.show()
